## Домашнее задание 3: Contingency Table

### О задании

1) Consider three nominal features (one of them, not more, may be taken from nominal features in your data).

2) Build two contingency tables over them: present a conditional frequency table and Quetelet relative index tables. Make comments on relations between categories of the common (to both tables) feature and two others.

3) Compute and visualize the chi-square-summary_Quetelet_index over both tables.

4) Comment on the meaning of the values in the data analysis context.

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

Загрузим данные

In [95]:
df = pd.read_csv('../../data/Vehicles03.csv')
df.head()

,Vehicle_Type,Sex_of_Driver,Age_of_Driver
0,9,2,74.0
1,11,1,42.0
2,11,1,35.0
3,9,1,62.0
4,9,2,49.0


Добавим 2 номинальных признака

In [96]:
# Motorcylce, vehilce type should be >=2 and <= 5
df = df.assign(Is_Motorcylce=df.Vehicle_Type.copy())
df["Is_Motorcylce"] = 1
df.loc[df["Vehicle_Type"]<2, "Is_Motorcylce"] = 0
df.loc[df["Vehicle_Type"]>5, "Is_Motorcylce"] = 0
df.loc[df["Vehicle_Type"]==-1, "Is_Motorcylce"] = 0

del df["Vehicle_Type"]

# Driver is younger than 40
df = df.assign(Age_of_Driver_LEQ40=df.Age_of_Driver.copy())
df.loc[df["Age_of_Driver"]<=40, "Age_of_Driver_LEQ40"] = 1
df.loc[df["Age_of_Driver"]>40, "Age_of_Driver_LEQ40"] = 0
df.head()

del df["Age_of_Driver"]
df.head()

,Sex_of_Driver,Is_Motorcylce,Age_of_Driver_LEQ40
0,2,0,0.0
1,1,0,0.0
2,1,0,1.0
3,1,0,0.0
4,2,0,0.0


In [97]:
df.columns

Index(['Sex_of_Driver', 'Is_Motorcylce', 'Age_of_Driver_LEQ40'], dtype='object')

В качестве 3-х номинальных признаков выберем: "Sex_of_Driver", "Is_Motorcylce", "Age_of_Driver_LEQ40"

In [98]:
import tabletext
# Для удобства построим таблицы cross classification

a = [["Sex_of_Driver/Is_Motorcylce", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]

total = df.shape[0] * df.shape[1]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Is_Motorcylce"] == j)).sum()
        a[i].append(x)

print(tabletext.to_text(a))

a = [["Sex_of_Driver/Age_of_Driver_LEQ40", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Age_of_Driver_LEQ40"] == j)).sum()
        a[i].append(x)

print(tabletext.to_text(a))

┌─────────────────────────────┬───────┬───────┐
│ Sex_of_Driver/Is_Motorcylce │ True  │ False │
├─────────────────────────────┼───────┼───────┤
│ Man                         │ 24406 │  4816 │
├─────────────────────────────┼───────┼───────┤
│ Woman                       │  9521 │   430 │
├─────────────────────────────┼───────┼───────┤
│ Unknown                     │    34 │     2 │
└─────────────────────────────┴───────┴───────┘
┌───────────────────────────────────┬───────┬───────┐
│ Sex_of_Driver/Age_of_Driver_LEQ40 │ True  │ False │
├───────────────────────────────────┼───────┼───────┤
│ Man                               │ 10730 │ 18492 │
├───────────────────────────────────┼───────┼───────┤
│ Woman                             │  3610 │  6341 │
├───────────────────────────────────┼───────┼───────┤
│ Unknown                           │    12 │    24 │
└───────────────────────────────────┴───────┴───────┘


Построим conditional frequency table

In [99]:
a = [["Sex_of_Driver/Is_Motorcylce", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]

total = df.shape[0]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Is_Motorcylce"] == j)).sum()
        a[i].append(str(x / total * 100)[0:8])

        
def copy_values(a):
    a1 = a[1:]
    for i in range(len(a1)):
        a1[i] = a1[i][1:]
        for j in range(len(a1[i])):
            a1[i][j] = float(a1[i][j])
    return a1

a1 = copy_values(a)
        
print(tabletext.to_text(a))

┌─────────────────────────────┬──────────┬──────────┐
│ Sex_of_Driver/Is_Motorcylce │ True     │ False    │
├─────────────────────────────┼──────────┼──────────┤
│ Man                         │ 62.24591 │ 12.28289 │
├─────────────────────────────┼──────────┼──────────┤
│ Woman                       │ 24.28269 │ 1.096686 │
├─────────────────────────────┼──────────┼──────────┤
│ Unknown                     │ 0.086714 │ 0.005100 │
└─────────────────────────────┴──────────┴──────────┘


Отсюда видно, что мужчин, попадающих в аварию больше жещин, но это обусловливается тем, что мужчин за рулем в принципе
 больше женщин. Рассматривая аварии на мотоциклах, видим, что большинство аварий совершается именно за рулем мотобайка.
Причем мужчины попадают в аварию гараздо чаще за рулем мотобайка, в более 60% случаях в аварию попадала именно такая категория(мужчина + мотобайк), а именно в 62.24591%. Касательно женщин можно сделать вывод, что почти все аварии с их участием случались за рулем мотобайка, а именно 24.28% на мотобайке против 1.09% не на мотобайке, относительно всех аварий.


In [100]:
a = [["Sex_of_Driver/Age_of_Driver_LEQ40", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Age_of_Driver_LEQ40"] == j)).sum()
        a[i].append(str(x / total * 100)[0:8])

a2 = copy_values(a)
        
print(tabletext.to_text(a))

┌───────────────────────────────────┬──────────┬──────────┐
│ Sex_of_Driver/Age_of_Driver_LEQ40 │ True     │ False    │
├───────────────────────────────────┼──────────┼──────────┤
│ Man                               │ 27.36616 │ 47.16264 │
├───────────────────────────────────┼──────────┼──────────┤
│ Woman                             │ 9.207069 │ 16.17230 │
├───────────────────────────────────┼──────────┼──────────┤
│ Unknown                           │ 0.030605 │ 0.061210 │
└───────────────────────────────────┴──────────┴──────────┘


Рассматривая данную табличку, напришивается весьма логичный вывод, что более старые люди попадают в аварию чаще молодых.
Об этом нам говорят полученные в ячейках числа. Возраст почти 2/3 мужчин, попавших в аварию превышает 40 лет, у жещин же
ситуация аналогична, ведь 27/46=0.58 почти равно 9/16=0.56, отсюда напришивается вывод, что только треть аварий
совершается людьми не старше 40 лет, будем их далее называть "молодыми", а остальных "старыми".

Построим Quetelet relative index table

In [101]:
a = [["Sex_of_Driver/Is_Motorcylce", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Is_Motorcylce"] == j)).sum() / total
        x /= (df["Sex_of_Driver"] == i).sum() / total
        y = (df["Is_Motorcylce"] == j).sum() / total
        a[i].append(str(((x - y) / y) * 100)[0:8])

a3 = copy_values(a)        

print(tabletext.to_text(a))

┌─────────────────────────────┬──────────┬──────────┐
│ Sex_of_Driver/Is_Motorcylce │ True     │ False    │
├─────────────────────────────┼──────────┼──────────┤
│ Man                         │ -3.57448 │ 23.13130 │
├─────────────────────────────┼──────────┼──────────┤
│ Woman                       │ 10.46409 │ -67.7155 │
├─────────────────────────────┼──────────┼──────────┤
│ Unknown                     │ 9.038962 │ -58.4931 │
└─────────────────────────────┴──────────┴──────────┘


Таблица Quetelet relative index относительно Is_Motorcycle показала, что на самом деле наличие за рулем женщины в аварии увеличивает частоту того, она будет за рулем мотобайка на 10% и уменьшает на 67% частоту того, что женщина попала в аварию не на мотобайке на 67%. С другой стороны, наличие за рулем мужчины влияет на рост обратным образом. Таким образом видно, что наличие за рулем определенного пола прямо противоположно влияет на рост частоты наличия байка в аварии.

In [102]:
a = [["Sex_of_Driver/Age_of_Driver_LEQ40", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Age_of_Driver_LEQ40"] == j)).sum() / total
        x /= (df["Sex_of_Driver"] == i).sum() / total
        y = (df["Age_of_Driver_LEQ40"] == j).sum() / total
        a[i].append(str(((x - y) / y) * 100)[0:8])

a4 = copy_values(a)
        
print(tabletext.to_text(a))

┌───────────────────────────────────┬──────────┬──────────┐
│ Sex_of_Driver/Age_of_Driver_LEQ40 │ True     │ False    │
├───────────────────────────────────┼──────────┼──────────┤
│ Man                               │ 0.314364 │ -0.18150 │
├───────────────────────────────────┼──────────┼──────────┤
│ Woman                             │ -0.89083 │ 0.514352 │
├───────────────────────────────────┼──────────┼──────────┤
│ Unknown                           │ -8.93487 │ 5.158841 │
└───────────────────────────────────┴──────────┴──────────┘


Таблица Quetelet relative index относительно "Age_of_Driver_LEQ40" говорит то, что и ожидалось, все ячейки отражают
приблизительно одинаковый и очень малый прирост ~=0.5%, что говорит о том, что пол почти не влияет на возраст водителя в аварии.

Теперь пострим хи-квадрат для "Age_of_Driver_LEQ40"

In [103]:
def mmul(a, b):
    c = a.copy()
    for i in range(len(a)):
        for j in range(len(a[i])):
            c[i][j] = a[i][j] * b[i][j] / 100
    return c

a = [["Sex_of_Driver/Is_Motorcylce", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]

b = mmul(a1, a3)
j = 1
for i in b:
    a[j] += i
    j += 1

print(tabletext.to_text(a))

┌─────────────────────────────┬──────────────────┬────────────────────────┐
│ Sex_of_Driver/Is_Motorcylce │ True             │ False                  │
├─────────────────────────────┼──────────────────┼────────────────────────┤
│ Man                         │  -2.224967603768 │          2.84119213457 │
├─────────────────────────────┼──────────────────┼────────────────────────┤
│ Woman                       │   2.540962536021 │         -0.74262640833 │
├─────────────────────────────┼──────────────────┼────────────────────────┤
│ Unknown                     │ 0.00783804550868 │ -0.0029831480999999997 │
└─────────────────────────────┴──────────────────┴────────────────────────┘


Таблица хи-квадрат показывает, на сколько сильно связаны признаки между собой(среднюю степень взаимосвязи). По данной таблице видно, что признаки "Sex_of_Driver" и "Is_Motorcylce" связаны очень слабо, так как все значения по абсолютной величине не больше 2-х.

Теперь пострим хи-квадрат для "Is_Motorcylce"

In [104]:
a = [["Sex_of_Driver/Age_of_Driver_LEQ40", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
b = mmul(a2, a4)
j = 1
for i in b:
    a[j] += i
    j += 1

print(tabletext.to_text(a))

┌───────────────────────────────────┬──────────────────────┬─────────────────┐
│ Sex_of_Driver/Age_of_Driver_LEQ40 │ True                 │ False           │
├───────────────────────────────────┼──────────────────────┼─────────────────┤
│ Man                               │  0.08602935522239999 │   -0.0856001916 │
├───────────────────────────────────┼──────────────────────┼─────────────────┤
│ Woman                             │ -0.08201933277270002 │  0.083182548496 │
├───────────────────────────────────┼──────────────────────┼─────────────────┤
│ Unknown                           │     -0.0027345169635 │ 0.0031577265761 │
└───────────────────────────────────┴──────────────────────┴─────────────────┘


А здесь значения крайне малы, как и ожидалось исходя из выводов по предыдущим таблицам, что пол и возраст почти никак не связаны в контексте аварий.